In [56]:
import time
import os
import pandas as pd
import datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [ ]:
LIST = 'daftar_anomali.xlsx'
SPV = 'wilayah_tugas_sak_82024.xlsx'

In [19]:
# Set download directory path
download_dir = os.getcwd()+r'\download'

# Create Chrome options object
chrome_options = Options()

# Set download directory preference
prefs = {'download.default_directory': download_dir}
chrome_options.add_experimental_option('prefs', prefs)

In [3]:
def check_button(xpath):
    try:
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath))
        )
        button.click()
    except:
        pass

In [20]:
driver = webdriver.Chrome(
    service=Service('chromedriver-win64/chromedriver.exe'),
    options=chrome_options
)

In [10]:
# PUSAT
list_error = pd.read_excel(LIST)

In [21]:
driver.get('https://fasih-dashboard.bps.go.id/superset/sqllab')

# Manual Login

In [33]:
for index, row in list_error.iterrows():
    driver.get(row['link'])
    time.sleep(1)

    check_button('//*[@id="js-sql-toolbar"]/div[1]/span[1]/span/button')
    time.sleep(1)

    while True:
        try:
            pending = driver.find_element(
                By.XPATH,
                ('/html/body/div[2]/div/div/div[2]/div/div/div/div[3]/div[3]'
                 '/div/div[2]/div/div[1]/div/span')
            ).text
            time.sleep(1)

            if pending != 'pending':
                break

        except:
            break
    time.sleep(1)

    while True:
        try:
            wait = driver.find_element(
                By.XPATH,
                ('/html/body/div[2]/div/div/div[2]/div/div/div/div[3]/div[3]'
                 '/div/div[2]/div/div[1]/div/div[3]/div/div/div[1]')
            ).text
            time.sleep(1)

            if wait != 'Running statement 1 out of 1':
                break

        except:
            break
    time.sleep(3)

    try:
        test = driver.find_element(
            By.XPATH,
            ('/html/body/div[2]/div/div/div[2]/div/div[1]/div/div[3]/div[3]'
             '/div/div[2]/div/div[1]/div/div[1]/div')
        ).text

        if test != 'The query returned no data':
            check_button(
                '/html/body/div[2]/div/div/div[2]/div/div[1]/div/div[3]'
                '/div[3]/div/div[2]/div/div[1]/div/div[1]/div'
            )
            time.sleep(1)

            filename = max([
                download_dir + '\\' + f for f in os.listdir(
                    download_dir
            )], key=os.path.getctime)
            df = pd.read_csv(filename, dtype='str')

            df['kode'] = row['kode']
            df['keterangan'] = row['keterangan']

            df = df[['kode', 'keterangan']+list(df.columns[:-2])]
            df.to_csv(filename, index=False)

    except:
        pass

    check_button(
        '//*[@id="a11y-query-editor-tabs"]/div[1]/div[1]/div/div[1]'
        '/button/span'
    )
    time.sleep(1)

# Join with the person in charge

In [53]:
df = pd.DataFrame()
for file in os.listdir('download'):
    temp = pd.read_csv(f'download/{file}', dtype='str')
    temp['Variable'] = temp[temp.columns[15:-1]].apply(
        lambda x: ', '.join(
            f'{col}:{val}' for col, val in zip(x.index, x.values)
        ), axis=1
    )
    temp = temp[[*temp.columns[:15], temp.columns[-1]]]

    df = pd.concat([df, temp])

df['idbs'] = (df['KODE_PROV'].astype(str) + df['KODE_KAB'] + df['KODE_KEC']
              + df['KODE_DESA'] + df['BS'])

In [46]:
spv = pd.read_excel(SPV)
spv['pml'] = spv['pml'].str.split(',').str[0]

In [54]:
final = pd.merge(df, spv)

In [ ]:
if not os.path.exists('anomali_petugas/'):
    os.makedirs('anomali_petugas/')

In [55]:
for i in final['pml'].unique():
    writer = pd.ExcelWriter(f'anomali_petugas/Anomali_{i}.xlsx')
    final.loc[final['pml']==i].to_excel(
        writer,
        sheet_name='Sheet1',
        index=False,
        na_rep='NaN'
    )

    for column in df:
        column_length = max(
            df[column].astype(str).map(len).max(),
            len(column)
        )+5

        col_idx = df.columns.get_loc(column)
        writer.sheets['Sheet1'].set_column(col_idx, col_idx, column_length)

    writer.close()

In [57]:
date = datetime.date.today().strftime("%Y-%m-%d")
final.to_excel(f'anomali_sak_{date}.xlsx', index=False)